In [6]:
import re
import os
import nltk
import string
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from pymongo import MongoClient
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import RegexpTokenizer 
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
nltk.download('stopwords')
import spacy
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package wordnet to /home/meteng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/meteng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
load_dotenv()

DATABASE_NAME = os.getenv("DATABASE_NAME")
DATABASE_PASSWORD = os.getenv("DATABASE_PASSWORD")

client = MongoClient("mongodb+srv://martin:{}@cluster0.ehkpp.mongodb.net/{}?retryWrites=true&w=majority".format(DATABASE_PASSWORD,DATABASE_NAME))

#client = MongoClient()
db = client.get_database(DATABASE_NAME)

lectures = db.lecture_profiles

In [13]:
ps = PorterStemmer() 
lemmatizer = WordNetLemmatizer() 
stop_words = set(stopwords.words('english')) 
tokenizer = RegexpTokenizer(r'\w+')
df = pd.DataFrame(list(lectures.find()))
data=df.drop(columns = ['_id', 'avg_rating', 'assigned_people', 'language', 'course_format', 'study_courses', 'positive_comments', 'negative_comments', 'semester', 'comments'])

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def remove_html(text):
    soup = BeautifulSoup(text, 'lxml')
    html_free = soup.get_text()
    
    return html_free

def remove_punctuation(text):
    no_punct = ''.join([c for c in text if c not in string.punctuation])
    return no_punct

def remove_stop_word(text):

    no_stop_word = [c for c in text if c not in stop_words]
    
    return no_stop_word

def word_lemmatizer(text):
    wordnet_tagged = map(lambda x: (x[0], get_wordnet_pos(x[1])), text)
    lemmatized_sentence = []
        
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return lemmatized_sentence

def word_lemmatizer2(text):
    lem_text = [ lemmatizer.lemmatize(i[0]) for i in text]
    return lem_text
    
def word_stemmer(text):
    stem_text = [ps.stem(i[0]) for i in text]
    return stem_text

def clean_data(data):
    html_free_data = data.apply(lambda x: remove_html(x))
    no_punct_data = html_free_data.apply(lambda x: remove_punctuation(x))
    tokenized_data = no_punct_data.apply(lambda x: tokenizer.tokenize(x.lower()))
    no_stop_word_data = tokenized_data.apply(lambda x: remove_stop_word(x))         
    tagged_data = no_stop_word_data.apply(lambda x: nltk.pos_tag(x))
    lemmatized_data = tagged_data.apply(lambda x: word_lemmatizer2(x))

    return lemmatized_data

for col in data.columns:
    print(col)

data['description'] = clean_data(data['description'])
data['learning_targets'] = clean_data(data['learning_targets'])
data['pre_qualifications'] = clean_data(data['pre_qualifications'])

#data['description'] = data['description'].apply(lambda x: word_stemmer(x))
"""
for (columnName, columnData) in data['pre_qualifications'].iteritems():
   print('Colunm Name : ', columnName)
   print('Column Contents : ', columnData)
"""
data.head()

name
description
learning_targets
pre_qualifications
lecture_id


,name,description,learning_targets,pre_qualifications,lecture_id
0,Learning Analytics,"[learning, analytics, la, attracted, great, de...","[successfully, completing, module, student, kn...",[],2
1,"Cloud, Web & Mobile","[lecture, present, theoretical, practical, asp...","[student, know, understand, architecture, algo...",[],3
2,Distributed Systems,"[lecture, present, important, concept, protoco...","[student, know, principle, protocol, algorithm...",[],4
3,Natural-Language-based Human-Computer Interaction,"[content, detail, level, language, analysis, l...","[student, learn, problem, occur, naturallangua...",[],5
4,Advanced Web Technologies,"[world, wide, web, underlying, technology, inc...","[successfully, completing, module, student, kn...","[basic, knowledge, web, technology, requiered]",6


In [ ]:
"""   
input_text = 'I am passing the input sentence here. so we will see what happens with this and.'

stop_words = set(stopwords.words('english')) 
word_tokens = word_tokenize(input_text) 

#print("studies:", lemmatizer.lemmatize("studies")) 
#print("corpora :", lemmatizer.lemmatize("corpora")) 
  
#print("better :", lemmatizer.lemmatize("better", pos ="n"))

def clean_text(text):#will replace the html characters with " "
    text=re.sub('<.*?>', ' ', text)  
    #To remove the punctuations
    text = text.translate(str.maketrans(' ',' ',string.punctuation))
    #will consider only alphabets and numerics
    text = re.sub('[^a-zA-Z]',' ',text)  
    #will replace newline with space
    text = re.sub("\n"," ",text)
    #will convert to lower case
    text = text.lower()
    # will split and join the words
    text=' '.join(text.split())
    return text#Running the Funtion

new_text=clean_text(input_text)
"""